### Algoritmo de Deutsch

O problema é verificar se determinar função $f(x)$ é constante ou balanceada.

Constante:

x | f | f
-- |-- | --
0  | 0 | 1
1  | 0 | 1


Balanceada:

x | f | f
--|-- | --
0 | 0 | 1
1 | 1 | 0


No clássico, para termos certeza dessa afirmação, teríamos que testar para os valores de $x = \{0,1\}$, para termos a conlusão, testando assim para cada possível entrada.

Na computação quântica, esse problema é resolvido pelo Algoritmo de Deutsch, representado pelo seguinte circuito:

![](Deutsch_algorithm_circuit.png)

Onde a operação representada pelo operador $U_{f}$ será nosso oráculo que irá operar sobre os qubits, avaliando como um todo nossa $f$, pois se aproveita do paralelismo quântico, assim podemos com 1 (uma) única medida afirmar se a função é balanceada ou constante.

Para caso $M(q_{1})=0$ a função é CONSTANTE, caso $M(q_{1})=1$ a função é BALANCEADA. Sendo $q_{1}$ o primeiro qubit do circuito.


Porém, foi verificado que a inclusão de $q_{2}$ nesse circuito é considerada redundante, podendo ser simplificado para o seguinte circuito abaixo, chamado Minimal Deutsch Algorithm, que utiliza um único qubit ([2012](https://www.researchgate.net/publication/235476206_Double-slit_implementation_of_the_minimal_Deutsch_algorithm)):

![](Deutsch_algorithm_circuit_simple.png)

Usando desse circuito vamos realizar a implementação do mesmo utilizando o Q\#:

Primeiro, vamos definir nossos oráculos, sendo estes os oráculo para as 4 funções possíveis nesse circuito:

![](oraculos.png)


### Testando o Algoritmo de Deutsch Simplificado

Para testarmos o circuito, vamos implementá-lo junto com os oráculos e testá-lo:

In [1]:
// Incluir as bibliotecas necessárias
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Measurement;

/snippet_.qs(2,6): warning QS6003: The namespace is already open.


 - Identidade: $U_{00}$ [CONSTANTE]

In [2]:
// Identidade
operation Oracle_I(x : Qubit) : Unit{
    I(x);
}

Oracle_I

 - Pauli Z: $U_{01}$ [BALANCEADA]

In [3]:
// Pauli_Z
operation Oracle_Z(x : Qubit) : Unit{
    Z(x);
}

Oracle_Z

 - \- Identidade: $U_{11}$ [CONSTANTE]

In [4]:
// Menos Identidade
// Na identidade negativa, vamos aplicar uma mudança de fase global de -1 no estado, 
// para isso usamos a operação R, que irá rotacionar o eixo.
open Microsoft.Quantum.Math;
operation Oracle_Menos_I(x : Qubit) : Unit{
    R(PauliI, 2.0 * PI(), x);
}

Oracle_Menos_I

 - \- Pauli Z: $U_{10}$ [BALANCEADA]

In [5]:
// Menos PauliZ
open Microsoft.Quantum.Math;
operation Oracle_Menos_Z(x : Qubit) : Unit{
    Z(x);
    R(PauliI, 2.0 * PI(), x);
}

Oracle_Menos_Z

Agora a implementação do circuito:

In [6]:
operation MinimalDeutschAlgorithm(oracle : (Qubit => Unit)) : Unit{
        using (q = Qubit()){
        
            H(q);
            oracle(q);
            H(q);

            if(M(q) == Zero){
                Message("Is constant!");
            }
            else{
                Message("Is balanced!");
            }
            Reset(q);
        }
    }

MinimalDeutschAlgorithm

Vamos aos testes:

In [7]:
operation Testes() : Unit{
    Message("----------------------");
    Message("Identidade: ");
    MinimalDeutschAlgorithm(Oracle_I);
    Message("----------------------");
    Message("PauliZ: ");
    MinimalDeutschAlgorithm(Oracle_Z);
    Message("----------------------");
    Message("-Identidade: ");
    MinimalDeutschAlgorithm(Oracle_Menos_I);
    Message("----------------------");
    Message("-PauliZ: ");
    MinimalDeutschAlgorithm(Oracle_Menos_Z);
    Message("----------------------");
}

Testes

In [8]:
%simulate Testes

----------------------
Identidade: 
Is constant!
----------------------
PauliZ: 
Is balanced!
----------------------
-Identidade: 
Is constant!
----------------------
-PauliZ: 
Is balanced!
----------------------


()

Podemos concluir, que obtivemos os resultados esperados utilizando este circuito, e com apenas 1 (uma) medida foi realmente possível determinar se a função é constante ou balanceada.

### Algoritmo de Deutsch

Para ilustrar, vamos recriar a operação para o algoritmo incluindo o segundo qubit.

![](Deutsch_algorithm_circuit.png)


In [14]:
operation DeutschAlgorithm(oracle : (Qubit[] => Unit)) : Unit{
        using (q = Qubit[2]){
        
            H(q[0]);
            H(q[1]);
            oracle(q);
            H(q[0]);

            if(M(q[0]) == Zero){
                Message("Is constant!");
            }
            else{
                Message("Is balanced!");
            }
            ResetAll(q);
        }
    }

DeutschAlgorithm

In [15]:
open Microsoft.Quantum.Math;
// Identidade
operation Oracle_I2(x : Qubit[]) : Unit{
    I(x[0]);
}
// Pauli_Z
operation Oracle_Z2(x : Qubit[]) : Unit{
    Z(x[0]);
}
// Menos Identidade
operation Oracle_Menos_I2(x : Qubit[]) : Unit{
    R(PauliI, 2.0 * PI(), x[0]);
}
// Menos PauliZ
operation Oracle_Menos_Z2(x : Qubit[]) : Unit{
    Z(x[0]);
    R(PauliI, 2.0 * PI(), x[0]);
}

Oracle_I2, Oracle_Menos_I2, Oracle_Menos_Z2, Oracle_Z2

In [16]:
operation Testes() : Unit{
    Message("----------------------");
    Message("Identidade: ");
    DeutschAlgorithm(Oracle_I2);
    Message("----------------------");
    Message("PauliZ: ");
    DeutschAlgorithm(Oracle_Z2);
    Message("----------------------");
    Message("-Identidade: ");
    DeutschAlgorithm(Oracle_Menos_I2);
    Message("----------------------");
    Message("-PauliZ: ");
    DeutschAlgorithm(Oracle_Menos_Z2);
    Message("----------------------");
}

Testes

In [17]:
%simulate Testes

----------------------
Identidade: 
Is constant!
----------------------
PauliZ: 
Is balanced!
----------------------
-Identidade: 
Is constant!
----------------------
-PauliZ: 
Is balanced!
----------------------


()

Observe que a saída é praticamente a mesma do algoritmo anterior.